# Part 3 - Text analysis and ethics

# 3.a Computing PMI

In this assessment you are tasked to discover strong associations between concepts in Airbnb reviews. The starter code we provide in this notebook is for orientation only. The below imports are enough to implement a valid answer.

### Imports, data loading and helper functions

We first connect our google drive, import pandas, numpy and some useful nltk and collections modules, then load the dataframe and define a function for printing the current time, useful to log our progress in some of the tasks.

In [245]:
import pandas as pd
from nltk.tag import pos_tag
from nltk import RegexpParser
import re
from collections import defaultdict,Counter
from nltk.stem import WordNetLemmatizer
from datetime import datetime
from tqdm import tqdm
import numpy as np
import os
tqdm.pandas()

In [246]:
# nltk imports, note that these outputs may be different if you are using colab or local jupyter notebooks
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\c2086876\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\c2086876\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\c2086876\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [247]:
# load stopwords
sw = set(stopwords.words('english'))

In [248]:
sw

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [249]:
'''We want to find the alternative forms of stopwords that have the "'" symbol in them 
in order to be able to add also to stopwords the word without this symbol'''

pattern = r'\w+\'\w+'

new_stopwords = []
for word in sw:
    # If it finds a word that contains "'" it appends the word in new_stopwords list
    if len(re.findall(pattern,word)) == 1:
        new_stopwords.append(re.findall(pattern,word)[0].replace('\'',''))
new_stopwords

['youre',
 'havent',
 'shouldnt',
 'wouldnt',
 'shant',
 'wasnt',
 'dont',
 'werent',
 'arent',
 'neednt',
 'didnt',
 'youve',
 'youd',
 'wont',
 'youll',
 'hadnt',
 'mightnt',
 'hasnt',
 'shes',
 'couldnt',
 'isnt',
 'shouldve',
 'doesnt',
 'thatll',
 'its',
 'mustnt']

In [250]:
# After checking those "new" words we add them to the stopwords variables named sw
for word in new_stopwords:
    sw.add(word)
sw

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'arent',
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'couldnt',
 'd',
 'did',
 'didn',
 "didn't",
 'didnt',
 'do',
 'does',
 'doesn',
 "doesn't",
 'doesnt',
 'doing',
 'don',
 "don't",
 'dont',
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'hadnt',
 'has',
 'hasn',
 "hasn't",
 'hasnt',
 'have',
 'haven',
 "haven't",
 'havent',
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'isnt',
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'mightnt',
 'more',
 'most',
 'mustn',
 "mustn't",
 'mustnt',
 'my',
 'myself',
 'needn',
 "needn't",
 'neednt',
 'no',
 'nor',
 'not',
 'no

In [251]:
basedir = os.getcwd()
df = pd.read_csv(os.path.join(basedir,'reviews.csv'))
# deal with empty reviews
df.comments = df.comments.fillna('')

In [252]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...


In [253]:
df.shape

(452143, 6)

### 3.a1 - Process reviews

What to implement: A `function process_reviews(df)` that will take as input the original dataframe and will return it with three additional columns: `tokenized`, `tagged` and `lower_tagged`.

In [254]:
def process_reviews(df):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
    # Initialize 3 lists one for each column we will create
    tokenized_col = []
    tagged_col = []
    lower_tagged_col = []


    mylen = len(df)
    count = 0
    
    # Iterate through the given dataframe
    for index, row in df.iterrows():
        # tokenize the words for the comments of a row
        token = word_tokenize(row.comments)
        # Append the tokenized words to the proper list
        tokenized_col.append(token)
        # Tag the tokenized words of the row and then append them to the proper list
        tagged_col.append(pos_tag(token))
        # lower_tagged.append(list(set(pos_tag([item.lower() for item in token]))))
        # Make the tagged words lowercased and then if they are not stopwords append them to the lower_tagged_col list
        lower_tagged_col.append(pos_tag([item.lower() for item in token if item.lower() not in sw]))
        count += 1

        if count % 1000 == 0:
            print(f'{count} out of {mylen}')

    # Set as values of the 3 new columns the proper list we created for each one
    df['tokenized'] = tokenized_col
    df['tagged'] = tagged_col
    df['lower_tagged'] = lower_tagged_col

    # Return the modified dataframe
    return df

In [255]:
df = process_reviews(df)
# df = process_reviews(df[:500])

1000 out of 452143
2000 out of 452143
3000 out of 452143
4000 out of 452143
5000 out of 452143
6000 out of 452143
7000 out of 452143
8000 out of 452143
9000 out of 452143
10000 out of 452143
11000 out of 452143
12000 out of 452143
13000 out of 452143
14000 out of 452143
15000 out of 452143
16000 out of 452143
17000 out of 452143
18000 out of 452143
19000 out of 452143
20000 out of 452143
21000 out of 452143
22000 out of 452143
23000 out of 452143
24000 out of 452143
25000 out of 452143
26000 out of 452143
27000 out of 452143
28000 out of 452143
29000 out of 452143
30000 out of 452143
31000 out of 452143
32000 out of 452143
33000 out of 452143
34000 out of 452143
35000 out of 452143
36000 out of 452143
37000 out of 452143
38000 out of 452143
39000 out of 452143
40000 out of 452143
41000 out of 452143
42000 out of 452143
43000 out of 452143
44000 out of 452143
45000 out of 452143
46000 out of 452143
47000 out of 452143
48000 out of 452143
49000 out of 452143
50000 out of 452143
51000 out

397000 out of 452143
398000 out of 452143
399000 out of 452143
400000 out of 452143
401000 out of 452143
402000 out of 452143
403000 out of 452143
404000 out of 452143
405000 out of 452143
406000 out of 452143
407000 out of 452143
408000 out of 452143
409000 out of 452143
410000 out of 452143
411000 out of 452143
412000 out of 452143
413000 out of 452143
414000 out of 452143
415000 out of 452143
416000 out of 452143
417000 out of 452143
418000 out of 452143
419000 out of 452143
420000 out of 452143
421000 out of 452143
422000 out of 452143
423000 out of 452143
424000 out of 452143
425000 out of 452143
426000 out of 452143
427000 out of 452143
428000 out of 452143
429000 out of 452143
430000 out of 452143
431000 out of 452143
432000 out of 452143
433000 out of 452143
434000 out of 452143
435000 out of 452143
436000 out of 452143
437000 out of 452143
438000 out of 452143
439000 out of 452143
440000 out of 452143
441000 out of 452143
442000 out of 452143
443000 out of 452143
444000 out of

### 3.a2 - Create a vocabulary

What to implement: A function `get_vocab(df)` which takes as input the DataFrame generated in step 1.c, and returns two lists, one for the 1,000 most frequent center words (nouns) and one for the 1,000 most frequent context words (either verbs or adjectives). 

In [256]:
def get_vocab(df):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
    cent_list, cont_list = [], []

    for review in df.lower_tagged:
        cent_list.extend([word for word in [list_of_words[0] for list_of_words in review if list_of_words[1][0] == 'N']])
        cont_list.extend([word for word in [list_of_words[0] for list_of_words in review if (list_of_words[1][0] == 'J') or (list_of_words[1][0] == 'V')]])

    cent_dict = Counter(cent_list)
    cont_dict = Counter(cont_list)

    cent_vocab = [key for key, value in sorted(cent_dict.items(), key=lambda item: item[1], reverse=True)][:1000]
    cont_vocab = [key for key, value in sorted(cont_dict.items(), key=lambda item: item[1], reverse=True)][:1000]

    return cent_vocab, cont_vocab

In [257]:
cent_vocab, cont_vocab = get_vocab(df)

In [258]:
cent_vocab[:5]

['place', 'apartment', 'location', 'stay', 'amsterdam']

In [259]:
cont_vocab[:5]

['great', 'nice', 'recommend', 'clean', 'good']

In [260]:
samewords = [name for name in cent_vocab if name in cont_vocab]
len(samewords)

385

In [261]:
samewords

['place',
 'apartment',
 'location',
 'stay',
 'amsterdam',
 'host',
 'home',
 'très',
 'center',
 '’',
 'walk',
 'centre',
 'tram',
 'experience',
 'à',
 'hosts',
 'neighborhood',
 'clean',
 'bien',
 'perfect',
 'la',
 'bed',
 'bathroom',
 'et',
 'e',
 'sehr',
 'thank',
 'breakfast',
 'lot',
 'ist',
 'street',
 'boat',
 'tips',
 'der',
 'visit',
 'coffee',
 'arrival',
 'bus',
 'need',
 'muy',
 'min',
 'appartement',
 'die',
 'que',
 'transport',
 'airbnb',
 'view',
 'cozy',
 'check',
 'shops',
 'kitchen',
 'bars',
 'minute',
 'helpful',
 'get',
 'es',
 'dans',
 'bit',
 'le',
 'lots',
 'je',
 'stairs',
 'con',
 'neighbourhood',
 'convenient',
 'zu',
 'du',
 'wir',
 'houseboat',
 'beautiful',
 'pictures',
 'des',
 'super',
 'train',
 'park',
 'il',
 'door',
 'couple',
 'supermarket',
 'séjour',
 'bike',
 'recommend',
 'metro',
 'è',
 'airport',
 'situé',
 'help',
 'casa',
 'friends',
 'alles',
 'balcony',
 'photos',
 'garden',
 'comfy',
 'appartment',
 'feel',
 'corner',
 'nice',
 'stop

### 3.a3 Count co-occurrences between center and context words

What to implement: A function `get_coocs(df, center_vocab, context_vocab)` which takes as input the DataFrame generated in step 1, and the lists generated in step 2 and returns a dictionary of dictionaries, of the form in the example above. It is up to you how you define context (full review? per sentence? a sliding window of fixed size?), and how to deal with exceptional cases (center words occurring more than once, center and context words being part of your vocabulary because they are frequent both as a noun and as a verb, etc). Use comments in your code to justify your approach. 

In [262]:
# def get_coocs(df, cent_vocab, cont_vocab):
#   sentences = []
#   comments = df.comments

#   for comment in comments:
#     sentences.extend([sentence for sentence in comment.split('.')])
  
#   print('yolo')
#   # print(sentences)
  
#   coocs = {}
  
#   count = 0
#   for center_word in cent_vocab:
#     count += 1
#     words = []
#     for sentence in sentences:
#       if center_word in sentence:
#         words_in_sentence = word_tokenize(sentence)
#         words.extend([word for word in words_in_sentence if word in cont_vocab])
    
#     center_word_dict = dict(Counter(words))
#     coocs[center_word] = center_word_dict
#     print(f'{count} out of 1000')
    
#   # cent_dict = Counter(cent_list)
#   # cont_dict = Counter(cont_list)

  
#   return coocs  

In [284]:
def get_coocs(df, cent_vocab, cont_vocab):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.
        cent_vocab: The dataframe we want to modify.
        cont_vocab: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
    sentences = []
    comments = df.comments

    start = pd.to_datetime('today')
    
    for comment in comments:
        sentences.extend([sentence for sentence in comment.split('.')])
        
    end = pd.to_datetime('today')
    diff = (end-start).total_seconds()
    
    print(f'yolo in {diff/60} minutes')
    
    start = pd.to_datetime('today')
  
    sentences_per_center_word = {center_word : Filter(sentences, center_word) for center_word in cent_vocab}

    end = pd.to_datetime('today')
    diff = (end-start).total_seconds()
    
    print(f'swag in {diff/60} minutes')

    
    coocs = {}

    count = 0
    count2 = 0
    diff = 0
    for center_word, sentences in sentences_per_center_word.items():
        words = []
        count += 1
        start = pd.to_datetime('today')
        count2 = 0
        
        for sentence in sentences:
            count2 += 1
#             words_of_sentence = [word for word in word_tokenize(sentence) if word in cont_vocab]
            words_of_sentence = [word for word in word_tokenize(sentence) if word in cont_vocab and word != center_word]
            if len(words_of_sentence) > 0: words.extend(words_of_sentence)
                
        end = pd.to_datetime('today')
        diff += (end-start).total_seconds()
#         print(f'{center_word} - - - - {(end-start).total_seconds()} seconds')
        coocs[center_word] = dict(Counter(words))
        if count % 50 == 0 : print(f'{count} center_word out of {len(sentences_per_center_word)} in {diff/60} minutes')

            
    print(diff/60)
    return coocs 

def Filter(sentences, center_word):
    sentences_for_center_word = []
    
    for sentence in sentences:
        if center_word in sentence.split():
            sentences_for_center_word.append(sentence)
#     print(f'{center_word} --- {len(sentences_for_center_word)}')
    return sentences_for_center_word

In [285]:
coocs = get_coocs(df, cent_vocab, cont_vocab)

yolo in 0.01371175 minutes
swag in 29.90134768333333 minutes
50 center_word out of 1000 in 10.251476966666669 minutes
100 center_word out of 1000 in 14.385687400000005 minutes
150 center_word out of 1000 in 17.387095833333337 minutes
200 center_word out of 1000 in 20.44182581666667 minutes
250 center_word out of 1000 in 21.77110443333333 minutes
300 center_word out of 1000 in 23.452622233333337 minutes
350 center_word out of 1000 in 24.697094650000004 minutes
400 center_word out of 1000 in 26.012559566666663 minutes
450 center_word out of 1000 in 26.98496018333333 minutes
500 center_word out of 1000 in 28.061973733333335 minutes
550 center_word out of 1000 in 28.789692616666667 minutes
600 center_word out of 1000 in 29.34931948333333 minutes
650 center_word out of 1000 in 29.768646766666667 minutes
700 center_word out of 1000 in 30.138031300000005 minutes
750 center_word out of 1000 in 30.99384561666667 minutes
800 center_word out of 1000 in 31.451505449999992 minutes
850 center_word o

In [ ]:
def get_coocs(df, cent_vocab, cont_vocab):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.
        cent_vocab: The dataframe we want to modify.
        cont_vocab: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
    sentences = []
    comments = df.comments

    start = pd.to_datetime('today')
    
    for comment in comments:
        sentences.extend([sentence for sentence in comment.split('.')])
        
    end = pd.to_datetime('today')
    diff = (end-start).total_seconds()
    
    print(f'yolo in {diff/60} minutes')
    
    start = pd.to_datetime('today')
  
    sentences_per_center_word = {center_word : Filter(sentences, center_word) for center_word in cent_vocab}

    end = pd.to_datetime('today')
    diff = (end-start).total_seconds()
    
    print(f'swag in {diff/60} minutes')

    
    coocs = {}

    count = 0
    count2 = 0
    diff = 0
    for center_word, sentences in sentences_per_center_word.items():
        words = []
        count += 1
        start = pd.to_datetime('today')
        count2 = 0
        
        for sentence in sentences:
            count2 += 1
#             words_of_sentence = [word for word in word_tokenize(sentence) if word in cont_vocab]
            words_of_sentence = [word for word in word_tokenize(sentence) if word in cont_vocab and word != center_word]
            if len(words_of_sentence) > 0: words.extend(words_of_sentence)
                
        end = pd.to_datetime('today')
        diff += (end-start).total_seconds()
#         print(f'{center_word} - - - - {(end-start).total_seconds()} seconds')
        coocs[center_word] = dict(Counter(words))
        if count % 50 == 0 : print(f'{count} center_word out of {len(sentences_per_center_word)} in {diff/60} minutes')

            
    print(diff/60)
    return coocs 

def Filter(sentences, center_word):
    sentences_for_center_word = []
    
    for sentence in sentences:
        if center_word in sentence.split():
            sentences_for_center_word.append(sentence)
#     print(f'{center_word} --- {len(sentences_for_center_word)}')
    return sentences_for_center_word

In [ ]:
coocs = get_coocs(df, cent_vocab, cont_vocab)

### 3.a4 Convert co-occurrence dictionary to 1000x1000 dataframe
What to implement: A function called `cooc_dict2df(cooc_dict)`, which takes as input the dictionary of dictionaries generated in step 3 and returns a DataFrame where each row corresponds to one center word, and each column corresponds to one context word, and cells are their corresponding co-occurrence value. Some (x,y) pairs will never co-occur, you should have a 0 value for those cases. 

In [289]:
def cooc_dict2df(coocs):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        coocs: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
    coocdf = pd.DataFrame(columns=cont_vocab, index = cent_vocab)

    for index, row in coocdf.iterrows():
        for word in cont_vocab:
            try:
                coocdf[word][index] = coocs[index][word]
            except: 
                coocdf[word][index] = 0

    return coocdf

In [290]:
coocdf = cooc_dict2df(coocs)
coocdf.shape

(1000, 1000)

### 3.a5 Raw co-occurrences to PMI scores

What to implement: A function `cooc2pmi(df)` that takes as input the DataFrame generated in step 4, and returns a new DataFrame with the same rows and columns, but with PMI scores instead of raw co-occurrence counts. 

In [291]:
coocdf

,great,nice,recommend,clean,good,stay,comfortable,easy,perfect,quiet,...,climb,chilled,downstairs,well-located,accommodate,based,andrea,avant,taxi,wi-fi
place,26420,14996,16646,14120,7276,36269,6695,4696,11603,6262,...,104,67,138,106,65,58,2,13,91,14
apartment,22344,15152,7999,19536,6362,16411,9446,5068,9495,6579,...,274,59,188,146,72,68,1,0,138,26
location,27734,8168,2605,6542,9425,6716,3286,5484,14100,6176,...,43,27,80,3,22,49,2,6,86,8
stay,22455,10532,15231,5843,5488,0,6791,3394,8952,2423,...,78,60,65,31,75,49,1,0,93,11
amsterdam,545,471,279,163,242,751,91,178,299,140,...,7,2,3,1,1,1,0,2,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
petits,0,0,2,1,0,0,0,0,2,0,...,0,0,0,0,0,0,0,7,2,0
show,215,154,31,97,75,136,46,51,49,20,...,0,1,3,0,1,2,0,0,7,0
découvrir,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9,0,1
wait,162,89,134,72,52,488,29,42,68,10,...,1,0,3,0,0,0,0,0,7,0


In [292]:
def cooc2pmi(df):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
    pmidf = pd.DataFrame(columns=cont_vocab, index = cent_vocab)

    N = 0
    for index, row in df.iterrows():
        N += sum(row)

    for index, row in df.iterrows():
        for word in cont_vocab:
            try:
                pmi = df[word][index] / (sum(df[word])/N / sum(row)/N)
                if pmi == 0:
                    pmidf[word][index] = 0
                else:
                    pmidf[word][index] = np.log([pmi])[0] 
                    #         print(pmidf[word][index])
            except: 
                pmidf[word][index] = 0
      
    return pmidf

In [293]:
pmidf = cooc2pmi(coocdf)
pmidf.shape

(1000, 1000)

In [294]:
pmidf

,great,nice,recommend,clean,good,stay,comfortable,easy,perfect,quiet,...,climb,chilled,downstairs,well-located,accommodate,based,andrea,avant,taxi,wi-fi
place,44.688360,44.561319,45.415802,44.752547,44.404542,45.314411,44.417571,43.974104,44.836389,44.397590,...,43.581010,44.346990,43.678172,44.995769,43.858426,44.256807,43.956814,41.736784,43.298902,43.361207
apartment,44.520872,44.571744,44.683024,45.077289,44.270379,44.521475,44.761877,44.050415,44.635966,44.447049,...,44.549823,44.219910,43.987436,45.316012,43.960780,44.415948,43.263743,0,43.715372,43.980322
location,44.353178,43.570040,43.177345,43.599478,44.279607,43.244221,43.322161,43.745508,44.647580,44.000041,...,42.314099,43.054414,42.749225,41.047222,42.391361,43.704465,43.573094,40.579874,42.858670,42.417871
stay,44.377028,44.059229,45.178240,43.721475,43.973800,0,44.283084,43.500678,44.428278,43.299373,...,43.144603,44.087918,42.776582,43.617593,43.852802,43.939460,43.114943,0,43.171917,42.971321
amsterdam,37.518569,37.811938,38.038388,37.002249,37.712443,38.148398,36.830614,37.412721,37.889113,37.308277,...,37.593829,37.546744,36.560830,37.043629,36.395338,36.907664,0,36.576281,37.291101,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
petits,0,0,32.389007,31.197182,0,0,0,0,32.170500,0,...,0,0,0,0,0,0,0,37.117727,35.481172,0
show,35.840567,35.946178,35.093309,35.735356,35.793139,35.691793,35.400541,35.414908,35.332635,34.614513,...,0,36.105743,35.812976,0,35.647484,36.852957,0,0,36.697398,0
découvrir,30.439857,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,37.302433,0,36.655523
wait,35.589380,35.429716,36.589016,35.469165,35.458749,37.001308,34.971050,35.252607,35.692177,33.953220,...,34.931919,0,35.844831,0,0,0,0,0,36.729252,0


### 3.a6 Retrieve top-k context words, given a center word

What to implement: A function `topk(df, center_word, N=10)` that takes as input: (1) the DataFrame generated in step 5, (2) a `center_word` (a string like `‘towels’`), and (3) an optional named argument called `N` with default value of 10; and returns a list of `N` strings, in order of their PMI score with the `center_word`. You do not need to handle cases for which the word `center_word` is not found in `df`. 

In [295]:
# pmidf.iloc[2,:]
pmidf['place']['room']

42.02694901656134

In [296]:
pmidf['great']['place']

44.68836008719678

In [297]:
pmidf['place']['nice']

43.888947057666115

In [298]:
pmidf['based']['place']

44.256807340404556

In [299]:
def topk(df, center_word, N=10):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.
        center_word: The dataframe we want to modify.
        N: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
    dicts_ = {word: df[word][center_word] for word in cont_vocab}
    top_words = [key for key, value in sorted(dicts_.items(), key=lambda item: item[1], reverse=True)][:N]

    return top_words

In [300]:
topk(pmidf, 'place')

['recommand',
 'recomend',
 'recommend',
 'recommendable',
 'stay',
 'reccomend',
 'maarten',
 'looking',
 'nicole',
 'affordable']

In [301]:
topk(pmidf, 'location')

['prime',
 'superb',
 'ideal',
 'convenient',
 'perfect',
 'terrific',
 'walkable',
 'brilliant',
 'fantastic',
 'central']

In [302]:
topk(pmidf, 'coffee')

['tea',
 'nespresso',
 'microwave',
 'complimentary',
 'fridge',
 'shops',
 'supplied',
 'nick',
 'cheese',
 'including']

In [303]:
topk(pmidf, 'stay')

['enjoyed',
 'enjoyable',
 'hesitate',
 'future',
 'memorable',
 'hope',
 'letting',
 'love',
 'longer',
 'pleasant']

In [304]:
topk(pmidf, 'petits')

['aux',
 'ses',
 'sont',
 'ont',
 'des',
 'apprécié',
 'hôtes',
 'les',
 'cafés',
 'avec']

In [305]:
topk(pmidf, 'sauber')

['zimmer',
 'sehr',
 'wie',
 'und',
 'alles',
 'wohnung',
 'ist',
 'zentral',
 'unterkunft',
 'allem']

# 3.b Ethical, social and legal implications



Local authorities in touristic hotspots like Amsterdam, NYC or Barcelona regulate the price of recreational apartments for rent to, among others, ensure that fair rent prices are kept for year-long residents. Consider your price recommender for hosts in Question 2c. Imagine that Airbnb recommends a new host to put the price of your flat at a price which is above the official regulations established by the local government. Upon inspection, you realize that the inflated price you have been recommended comes from many apartments in the area only being offered during an annual event which brings many tourists, and which causes prices to rise. 

In this context, critically reflect on the compliance of this recommender system with **one of the five actions** outlined in the **UK’s Data Ethics Framework**. You should prioritize the action that, in your opinion, is the weakest. Then, justify your choice by critically analyzing the three **key principles** outlined in the Framework, namely _transparency_, _accountability_ and _fairness_. Finally, you should propose and critically justify a solution that would improve the recommender system in at least one of these principles. You are strongly encouraged to follow a scholarly approach, e.g., with peer-reviewed references as support. 

Your report should be between 500 and 750 words long.  

### Your answer here. No Python, only Markdown.

Write your answer after the line.

---

...